In [2]:
from werkzeug.wrappers import Request, Response
#from flask import Flask, render_template
from jinja2 import Template

#from datetime import date
from datetime import datetime, timedelta

import collections
from amigocloud import AmigoCloud

## REPORTE
from docxtpl import DocxTemplate
import docxtpl
from docx.shared import Mm
import requests

In [3]:
acceso = open('../api_key/api.key','r')
api_token = acceso.readlines()[0]
acceso.close()

amigocloud = AmigoCloud(token=api_token)
amigocloud

In [4]:
def generar_reporte(i):
    id_reg = i
    
    # SELECCIONAR REGISTRO
    query = {'query': 'select * from dataset_296354 where id = {id_ca}'.format(id_ca=id_reg)}
    project = 'https://app.amigocloud.com/api/v1/projects/32082/sql'
    select = amigocloud.get(project, query)
    data = select['data'][0]
    data['fecha_de_registro'] = datetime.strptime(data['fecha_de_registro'][0:10], '%Y-%m-%d').strftime('%d/%m/%Y')
    #data['fecha_quema'] = datetime.strptime(data['fecha_quema'][0:10], '%Y-%m-%d').strftime('%d/%m/%Y')
    #data['fecha_inspeccion'] = datetime.strptime(data['fecha_inspeccion'][0:10], '%Y-%m-%d').strftime('%d/%m/%Y')
    insp = collections.namedtuple("insp", data.keys())(*data.values())
    print('CAÑERO: ' + insp.canhero)
    #FOTOS
    query_fotos = {'query': 'select source_amigo_id, s3_filename from gallery_43414 where source_amigo_id=\'{source_amigo_id}\''.format(source_amigo_id=insp.amigo_id)}
    project = 'https://app.amigocloud.com/api/v1/projects/32082/sql'
    fotos = amigocloud.get(project, query_fotos)['data']
    
    #GENERAR DOCX
    doc = DocxTemplate("templates/01_template_operador.docx")
    #descargar fotos y generar lista InlineImage
    lista_fotos_inline = []
    for foto in fotos:
        #response = requests.get("https://www-amigocloud.s3.amazonaws.com/gallery/" + foto['file_name'])
        response = requests.get("https://www-amigocloud.s3.amazonaws.com/gallery/" + foto['s3_filename'])
        file = open('fotos/' + foto['s3_filename'], "wb")
        file.write(response.content)
        file.close()
        lista_fotos_inline.append({'foto': docxtpl.InlineImage(doc, image_descriptor='fotos/' + foto['s3_filename'], width=Mm(50))})

    context = {'insp':insp, 'fotos':lista_fotos_inline}

    doc.render(context)

    # formato de nombre de archivo: "123_EVC_01-01-2022_NOMBRE"
    cod_nom = insp.canhero.split(' / ')
    file_name = cod_nom[0] + '_EVC_' + insp.fecha_de_registro.replace('/','-') + '_' + cod_nom[1] + '_' + str(insp.id)

    #doc.save(r'C:\Users\UTEA\OneDrive - Ingenio Azucarero Guabira S.A (2)\INFORMES\IMFORMES PARA REVISAR\\' + file_name + '.docx')
    doc.save(r'_' + file_name + '.docx')

In [5]:
lista_id = list(range(145, 151))
lista_id

[145, 146, 147, 148, 149, 150]

In [6]:
for i in lista_id:
    print('GENERANDO id = ' + str(i))
    generar_reporte(i)
print('END=====================================')

GENERANDO id = 145
CAÑERO: 7542 / LAZARTE PEREZ VICTOR
GENERANDO id = 146
CAÑERO: 639 / AGROPECUARIA SERVICOL
GENERANDO id = 147
CAÑERO: 594 / AGROPECUARIA FLAMBOYAN
GENERANDO id = 148
CAÑERO: 86 / AGUILERA TARADELLES JOSE LUIS
GENERANDO id = 149
CAÑERO: 9278 / NUNEZ MARTINEZ BRAULIO
GENERANDO id = 150
CAÑERO: 1741 / CARO ALTAMIRANO ROSARIO
END=====================================
